In [1]:
!wget https://hothienlac.s3-ap-southeast-1.amazonaws.com/flowprint/flowprint.zip

--2021-06-11 16:19:33--  https://hothienlac.s3-ap-southeast-1.amazonaws.com/flowprint/flowprint.zip
Resolving hothienlac.s3-ap-southeast-1.amazonaws.com (hothienlac.s3-ap-southeast-1.amazonaws.com)... 52.219.132.79
Connecting to hothienlac.s3-ap-southeast-1.amazonaws.com (hothienlac.s3-ap-southeast-1.amazonaws.com)|52.219.132.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36771 (36K) [application/zip]
Saving to: ‘flowprint.zip’

flowprint.zip       100%[===================>]  35.91K   167KB/s    in 0.2s    

2021-06-11 16:19:34 (167 KB/s) - ‘flowprint.zip’ saved [36771/36771]



In [2]:
!unzip flowprint.zip

Archive:  flowprint.zip
   creating: flowprint/
  inflating: flowprint/evaluation.py  
   creating: flowprint/flowprint/
  inflating: flowprint/flowprint/browser_detector.py  
  inflating: flowprint/flowprint/cluster.py  
  inflating: flowprint/flowprint/cross_correlation_graph.py  
  inflating: flowprint/flowprint/cross_correlation_graph1.py  
  inflating: flowprint/flowprint/fingerprint.py  
  inflating: flowprint/flowprint/fingerprints.py  
  inflating: flowprint/flowprint/flowprint.py  
  inflating: flowprint/flowprint/flows.py  
  inflating: flowprint/flowprint/flow_generator.py  
  inflating: flowprint/flowprint/network_destination.py  
  inflating: flowprint/flowprint/preprocessor.py  
  inflating: flowprint/flowprint/reader.py  
  inflating: flowprint/flowprint/recognition.py  
  inflating: flowprint/flowprint/test.py  
 extracting: flowprint/flowprint/__init__.py  
  inflating: flowprint/flowprint/__main__.py  
  inflating: flowprint/load_csv.py   
  inflating: flowprint/test_

In [3]:
!mv flowprint/* .

mv: cannot move 'flowprint/flowprint' to './flowprint': Directory not empty


In [4]:
!mv flowprint/flowprint/* flowprint

In [5]:
!rm -r flowprint/flowprint

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import json
from load_csv import load_csv
from flowprint.flowprint     import FlowPrint
import numpy as np
import operator
import random
from tqdm import tqdm

In [2]:
MODEL_PATH = '/content/drive/MyDrive/AI PROJECTS/GNN/flowprint/T4/t10/models/0'
TEST_DATA_PATH = '/content/drive/MyDrive/AI PROJECTS/GNN/dataset/4_2/test'
RESULT_FOLDER = '/content/drive/MyDrive/AI PROJECTS/GNN/flowprint/T4/t10/predicts'

MODELS_TO_TEST = os.listdir(MODEL_PATH)

In [3]:
def get_max_frequent(x):

    if len(x) == 0:
        return 'unknown_unknown'

    frequencies = {}
    for item in x:
        if item in frequencies:
            frequencies[item] += 1
        else:
            frequencies[item] = 1

    return max(frequencies.items(), key=operator.itemgetter(1))[0]

In [4]:
def predict(model_on_memory_file, app_sample):
    X, y = list(), list()

    app_path = f'{TEST_DATA_PATH}/{"_".join(app_sample.split("_")[:-2])}/{app_sample}'
    X_, y_ = load_csv(app_path, 'y')
    try:
        X = np.concatenate(X_)
        y = np.concatenate(y_)
    except Exception:
        X = np.array([], dtype=object)
        y = np.array([], dtype=object)

    model = FlowPrint()
    model.load_on_memory_file(model_on_memory_file)
    fp_test = model.fingerprinter.fit_predict(X)
    y_pred = [list(_)[0] for _ in model.recognize(fp_test)]
    prediction = '_'.join(get_max_frequent(y_pred).split('_')[:-2])

    return prediction

In [5]:
def test_model_with_app(model, app, model_name):
    predictions = []

    # app_samples_list = tqdm(
    #     [_ for _ in os.listdir(f'{TEST_DATA_PATH}/{app}') if _ != '.ipynb_checkpoints'],
    #     desc = f'[Model {model_name}] [App: {app}]'
    # )

    for app_sample in [_ for _ in os.listdir(f'{TEST_DATA_PATH}/{app}') if _ != '.ipynb_checkpoints']:
        prediction = predict(model, app_sample)
        predictions.append(prediction)

    return predictions

In [11]:
def test_model(model_name):
    if not os.path.exists(f'{RESULT_FOLDER}/{model_name}'):
        os.makedirs(f'{RESULT_FOLDER}/{model_name}')

    with open(f'{MODEL_PATH}/{model_name}', 'r') as model_file:

        model = json.load(model_file)

    app_list = [_ for _ in os.listdir(TEST_DATA_PATH) if _ != '.ipynb_checkpoints']
    for app in app_list:
        result_file = f'{RESULT_FOLDER}/{model_name}/{app}.json'
        if os.path.isfile(result_file):
            print(f'[ PASSED ]: Model {model_name}\t{app}\talready predicted')
            continue
        
        predictions = test_model_with_app(model, app, model_name)
        
        with open(result_file, 'w') as file:
            json.dump(
                {
                    'predictions': predictions,
                },
                file,
            )
        
        print(f'[ DONE ]: Model {model_name}\t{app}')

In [ ]:
for model in MODELS_TO_TEST:
    test_model(model)